# Wordsearch
Given a text file, consisting of three parts (a grid size, a grid, and a list of words), find:
* the words present in the grid, 
* the longest word present in the grid, 
* the number of words not present in the grid, 
* the longest word not present that can be formed from the leftover letters

The only words that need be considered are the ones given in the list in the puzzle input.

The puzzle consists of:
1. A line consisting of _w_`x`_h_, where _w_ and _h_ are integers giving the width and height of the grid.
2. The grid itself, consisting of _h_ lines each of _w_ letters.
3. A list of words, one word per line, of arbitrary length. 

## Example


￼`...p.mown.
.sdse..ee.
.e.elad.cr
pi.dtir.ah
rzsiwovspu
oawh.kieab
brow.c.rda
ecnotops.r
d.kc.d...b
.staple...`

```
fhjpamownq
wsdseuqeev
ieaeladhcr
piedtiriah
rzsiwovspu
oawhakieab
browpcfrda
ecnotopssr
dikchdnpnb
bstapleokr
```

14 words added;  6 directions

Present: apace cowhides crazies dock knows lived mown pears probed rhubarb rioted staple tops wide

Decoys: adapting bombing boor brick cackles carnal casino chaplets chump coaster coccyxes coddle collies creels crumbled cunt curds curled curlier deepen demeanor dicier dowses ensuing faddish fest fickler foaming gambol garoting gliding gristle grunts guts ibex impugns instants kielbasy lanyard loamier lugs market meanly minuend misprint mitts molested moonshot mucking oaks olives orgasmic pastrami perfect proceed puckered quashed refined regards retraces revel ridges ringlet scoff shinier siren solaria sprain sunder sunup tamped tapes thirds throw tiller times trains tranquil transfix typesets uric wariness welts whimsy winced winced

Decoys: fickler, adapting, chump, foaming, molested, carnal, crumbled, guts, minuend, bombing, winced, coccyxes, solaria, shinier, cackles

All words: adapting, apace, bombing, cackles, carnal, chump, coccyxes, cowhides, crazies, crumbled, dock, fickler, foaming, guts, knows, lived, minuend, molested, mown, pears, probed, rhubarb, rioted, shinier, solaria, staple, tops, wide, winced

Directions:  [('probed', '`(True, 3, 0, <Direction.down: 4>)`'), ('staple', '`(True, 9, 1, <Direction.right: 2>)`'), ('rioted', '`(True, 6, 7, <Direction.upleft: 5>)`'), ('cowhides', '`(True, 8, 3, <Direction.up: 3>)`'), ('tops', '`(True, 7, 4, <Direction.right: 2>)`'), ('knows', '`(True, 8, 2, <Direction.up: 3>)`'), ('lived', '`(True, 2, 4, <Direction.downright: 8>)`'), ('rhubarb', '`(True, 2, 9, <Direction.down: 4>)`'), ('crazies', '`(True, 7, 1, <Direction.up: 3>)`'), ('dock', '`(True, 8, 5, <Direction.up: 3>)`'), ('apace', '`(True, 5, 8, <Direction.up: 3>)`'), ('mown', '`(True, 0, 5, <Direction.right: 2>)`'), ('pears', '`(True, 0, 3, <Direction.downright: 8>)`'), ('wide', '`(True, 4, 4, <Direction.upright: 6>)`')]

In [4]:
import string
import re
import collections
import copy
import os

from enum import Enum
Direction = Enum('Direction', 'left right up down upleft upright downleft downright')
    
delta = {Direction.left: (0, -1),Direction.right: (0, 1), 
         Direction.up: (-1, 0), Direction.down: (1, 0), 
         Direction.upleft: (-1, -1), Direction.upright: (-1, 1), 
         Direction.downleft: (1, -1), Direction.downright: (1, 1)}

cat = ''.join
wcat = ' '.join
lcat = '\n'.join

In [5]:
def empty_grid(w, h):
    return [['.' for c in range(w)] for r in range(h)]

In [6]:
def show_grid(grid):
    return lcat(cat(r) for r in grid)

In [7]:
def indices(grid, r, c, l, d):
    dr, dc = delta[d]
    w = len(grid[0])
    h = len(grid)
    inds = [(r + i * dr, c + i * dc) for i in range(l)]
    return [(i, j) for i, j in inds
           if i >= 0
           if j >= 0
           if i < h
           if j < w]

In [8]:
def gslice(grid, r, c, l, d):
    return [grid[i][j] for i, j in indices(grid, r, c, l, d)]

In [9]:
def set_grid(grid, r, c, d, word):
    for (i, j), l in zip(indices(grid, r, c, len(word), d), word):
        grid[i][j] = l
    return grid

In [10]:
def present(grid, word):
    w = len(grid[0])
    h = len(grid)
    for r in range(h):
        for c in range(w):
            for d in Direction:
                if cat(gslice(grid, r, c, len(word), d)) == word:
                    return True, r, c, d
    return False, 0, 0, list(Direction)[0]

In [158]:
def present_many(grid, words):
    w = len(grid[0])
    h = len(grid)
    wordlens = set(len(w) for w in words)
    presences = []
    for r in range(h):
        for c in range(w):
            for d in Direction:
                for wordlen in wordlens:
                    word = cat(gslice(grid, r, c, wordlen, d))
                    if word in words:
                        presences += [(word, r, c, d)]
    return set(presences)

In [141]:
# def present_many(grid, words):
#     w = len(grid[0])
#     h = len(grid)
#     wordlens = set(len(w) for w in words)
#     presences = set()
#     for r in range(h):
#         for c in range(w):
#             for d in Direction:
#                 for wordlen in wordlens:
#                     word = cat(gslice(grid, r, c, wordlen, d))
#                     if word in words:
#                         presences.add((word, r, c, d))
#     return presences

In [11]:
def read_wordsearch(fn):
    lines = [l.strip() for l in open(fn).readlines()]
    w, h = [int(s) for s in lines[0].split('x')]
    grid = lines[1:h+1]
    words = lines[h+1:]
    return w, h, grid, words

In [49]:
width, height, g, ws = read_wordsearch('wordsearch04.txt')
g, ws

(['pistrohxegniydutslxt',
  'wmregunarbpiledsyuoo',
  'hojminbmutartslrlmgo',
  'isrsdniiekildabolpll',
  'tstsnyekentypkalases',
  'ssnetengcrfetedirgdt',
  'religstasuslatxauner',
  'elgcpgatsklglzistilo',
  'tndlimitationilkasan',
  'aousropedlygiifeniog',
  'kilrprepszffsyzqsrhs',
  'itlaadorableorpccese',
  'noaeewoodedpngmqicnl',
  'gmrtoitailingchelrok',
  'jadsngninetsahtooeic',
  'xeernighestsailarmtu',
  'aeabsolvednscumdfnon',
  'gydammingawlcandornk',
  'hurlerslvkaccxcinosw',
  'iqnanoitacifitrofqqi'],
 ['absolved',
  'adorable',
  'aeon',
  'alias',
  'ancestor',
  'baritone',
  'bemusing',
  'blonds',
  'bran',
  'calcite',
  'candor',
  'conciseness',
  'consequent',
  'cuddle',
  'damming',
  'dashboards',
  'despairing',
  'dint',
  'dullard',
  'dynasty',
  'employer',
  'exhorts',
  'feted',
  'fill',
  'flattens',
  'foghorn',
  'fortification',
  'freakish',
  'frolics',
  'gall',
  'gees',
  'genies',
  'gets',
  'hastening',
  'hits',
  'hopelessness',
  'hurler

In [50]:
for w in ws:
    print(w, present(g, w))

absolved (True, 16, 2, <Direction.right: 2>)
adorable (True, 11, 4, <Direction.right: 2>)
aeon (True, 11, 4, <Direction.down: 4>)
alias (True, 15, 15, <Direction.left: 1>)
ancestor (False, 0, 0, <Direction.left: 1>)
baritone (False, 0, 0, <Direction.left: 1>)
bemusing (False, 0, 0, <Direction.left: 1>)
blonds (False, 0, 0, <Direction.left: 1>)
bran (True, 1, 9, <Direction.left: 1>)
calcite (True, 19, 9, <Direction.upright: 6>)
candor (True, 17, 12, <Direction.right: 2>)
conciseness (False, 0, 0, <Direction.left: 1>)
consequent (False, 0, 0, <Direction.left: 1>)
cuddle (False, 0, 0, <Direction.left: 1>)
damming (True, 17, 2, <Direction.right: 2>)
dashboards (False, 0, 0, <Direction.left: 1>)
despairing (False, 0, 0, <Direction.left: 1>)
dint (False, 0, 0, <Direction.left: 1>)
dullard (True, 8, 2, <Direction.down: 4>)
dynasty (True, 3, 4, <Direction.downright: 8>)
employer (False, 0, 0, <Direction.left: 1>)
exhorts (True, 0, 8, <Direction.left: 1>)
feted (True, 5, 10, <Direction.right: 2

Which words are present?

In [69]:
%%timeit
[w for w in ws if present(g, w)[0]]

1 loop, best of 3: 1.28 s per loop


In [70]:
%%timeit
[p[0] for p in present_many(g, ws)]

1 loop, best of 3: 215 ms per loop


In [61]:
ws

['absolved',
 'adorable',
 'aeon',
 'alias',
 'ancestor',
 'baritone',
 'bemusing',
 'blonds',
 'bran',
 'calcite',
 'candor',
 'conciseness',
 'consequent',
 'cuddle',
 'damming',
 'dashboards',
 'despairing',
 'dint',
 'dullard',
 'dynasty',
 'employer',
 'exhorts',
 'feted',
 'fill',
 'flattens',
 'foghorn',
 'fortification',
 'freakish',
 'frolics',
 'gall',
 'gees',
 'genies',
 'gets',
 'hastening',
 'hits',
 'hopelessness',
 'hurlers',
 'impales',
 'infix',
 'inflow',
 'innumerable',
 'intentional',
 'jerkin',
 'justification',
 'kitty',
 'knuckles',
 'leaving',
 'like',
 'limitation',
 'locoweeds',
 'loot',
 'lucking',
 'lumps',
 'mercerising',
 'monickers',
 'motionless',
 'naturally',
 'nighest',
 'notion',
 'ogled',
 'originality',
 'outings',
 'pendulous',
 'piled',
 'pins',
 'pithier',
 'prep',
 'randomness',
 'rectors',
 'redrew',
 'reformulated',
 'remoteness',
 'retaking',
 'rethink',
 'rope',
 'rubier',
 'sailors',
 'scowls',
 'scum',
 'sepals',
 'sequencers',
 'serf',


What is the longest word that is present?

In [15]:
sorted([w for w in ws if present(g, w)[0]], key=len)[-1]

'fortification'

What is the longest word that is absent?

In [16]:
sorted([w for w in ws if not present(g, w)[0]], key=len)[-1]

'justification'

How many letters are unused?

In [17]:
g0 = empty_grid(width, height)
for w in ws:
    p, r, c, d = present(g, w)
    if p:
        set_grid(g0, r, c, d, w)
len([c for c in cat(cat(l) for l in g0) if c == '.'])

57

What is the longest word you can make form the leftover letters?

In [18]:
unused_letters = [l for l, u in zip((c for c in cat(cat(l) for l in g)), (c for c in cat(cat(l) for l in g0)))
                  if u == '.']
unused_letter_count = collections.Counter(unused_letters)
unused_letter_count

Counter({'a': 4,
         'b': 1,
         'c': 5,
         'd': 3,
         'e': 1,
         'g': 2,
         'i': 5,
         'j': 2,
         'k': 3,
         'l': 2,
         'm': 3,
         'n': 3,
         'p': 3,
         'q': 5,
         'r': 3,
         's': 3,
         'w': 2,
         'x': 4,
         'y': 2,
         'z': 1})

In [19]:
unused_words = [w for w in ws if not present(g, w)[0]]
unused_words

['ancestor',
 'baritone',
 'bemusing',
 'blonds',
 'conciseness',
 'consequent',
 'cuddle',
 'dashboards',
 'despairing',
 'dint',
 'employer',
 'freakish',
 'gall',
 'hopelessness',
 'impales',
 'infix',
 'inflow',
 'innumerable',
 'intentional',
 'jerkin',
 'justification',
 'leaving',
 'locoweeds',
 'monickers',
 'originality',
 'outings',
 'pendulous',
 'pithier',
 'randomness',
 'rectors',
 'redrew',
 'reformulated',
 'remoteness',
 'rethink',
 'scowls',
 'sequencers',
 'serf',
 'shook',
 'spottiest',
 'stood',
 'surtaxing',
 'wardrobes']

In [20]:
makeable_words = []
for w in unused_words:
    unused_word_count = collections.Counter(w)
    if all(unused_word_count[l] <= unused_letter_count[l] for l in unused_word_count):
        makeable_words += [w]
        print('*', end='')
    print(w, unused_word_count)

ancestor Counter({'t': 1, 'r': 1, 'a': 1, 'e': 1, 'o': 1, 's': 1, 'n': 1, 'c': 1})
baritone Counter({'t': 1, 'i': 1, 'a': 1, 'e': 1, 'n': 1, 'o': 1, 'r': 1, 'b': 1})
bemusing Counter({'m': 1, 'i': 1, 'n': 1, 'g': 1, 'e': 1, 's': 1, 'u': 1, 'b': 1})
blonds Counter({'n': 1, 's': 1, 'd': 1, 'l': 1, 'o': 1, 'b': 1})
conciseness Counter({'s': 3, 'n': 2, 'e': 2, 'c': 2, 'i': 1, 'o': 1})
consequent Counter({'n': 2, 'e': 2, 't': 1, 'q': 1, 's': 1, 'u': 1, 'o': 1, 'c': 1})
cuddle Counter({'d': 2, 'l': 1, 'u': 1, 'e': 1, 'c': 1})
dashboards Counter({'a': 2, 'd': 2, 's': 2, 'r': 1, 'h': 1, 'o': 1, 'b': 1})
*despairing Counter({'i': 2, 'a': 1, 'g': 1, 'e': 1, 'n': 1, 'r': 1, 'd': 1, 'p': 1, 's': 1})
dint Counter({'t': 1, 'i': 1, 'd': 1, 'n': 1})
employer Counter({'e': 2, 'm': 1, 'r': 1, 'p': 1, 'o': 1, 'l': 1, 'y': 1})
freakish Counter({'i': 1, 'a': 1, 'h': 1, 'e': 1, 's': 1, 'r': 1, 'f': 1, 'k': 1})
*gall Counter({'l': 2, 'a': 1, 'g': 1})
hopelessness Counter({'s': 4, 'e': 3, 'n': 1, 'h': 1, 'l':

In [21]:
max(len(w) for w in makeable_words)

10

In [22]:
sorted(makeable_words, key=len)[-1]

'despairing'

In [78]:
def do_wordsearch_tasks_old(fn, show_anyway=False):
    width, height, grid, words = read_wordsearch(fn)

    used_words = [w for w in words if present(grid, w)[0]]
    unused_words = [w for w in words if not present(grid, w)[0]]
    lwp = sorted([w for w in words if present(grid, w)[0]], key=len)[-1]
    lwa = sorted(unused_words, key=len)[-1]

    lwps = [w for w in used_words if len(w) == len(lwp)]
    lwas = [w for w in unused_words if len(w) == len(lwa)]
    g0 = empty_grid(width, height)
    for w in words:
        p, r, c, d = present(grid, w)
        if p:
            set_grid(g0, r, c, d, w) 
    unused_letters = [l for l, u in zip((c for c in cat(cat(l) for l in grid)), (c for c in cat(cat(l) for l in g0)))
                  if u == '.']
    unused_letter_count = collections.Counter(unused_letters)
    makeable_words = []
    for w in unused_words:
        unused_word_count = collections.Counter(w)
        if all(unused_word_count[l] <= unused_letter_count[l] for l in unused_word_count):
            makeable_words += [w]
    lwm = sorted(makeable_words, key=len)[-1]
    lwms = [w for w in makeable_words if len(w) == len(lwm)]
    if show_anyway or len(set((len(lwp),len(lwa),len(lwm)))) == 3:
        print('\n{}'.format(fn))
        print('{} words present'.format(len(words) - len(unused_words)))
        print('Longest word present: {}, {} letters ({})'.format(lwp, len(lwp), lwps))
        print('Longest word absent: {}, {} letters ({})'.format(lwa, len(lwa), lwas))
        print('{} unused letters'.format(len([c for c in cat(cat(l) for l in g0) if c == '.'])))
        print('Longest makeable word: {}, {} ({})'.format(lwm, len(lwm), lwms))
        print('All makeable words: {}'.format(makeable_words))

In [152]:
def do_wordsearch_tasks(fn, show_anyway=False, show_all_makeable=False):
    width, height, grid, words = read_wordsearch(fn)
    used_words = [p[0] for p in present_many(grid, words)]
    unused_words = [w for w in words if w not in used_words]
    lwp = max(used_words, key=len)
    lwps = [w for w in used_words if len(w) == len(lwp)]

    if unused_words:
        lwa = max(unused_words, key=len)
    
        lwas = [w for w in unused_words if len(w) == len(lwa)]
        unused_grid = [[c for c in r] for r in grid]
        for w, r, c, d in present_many(grid, words):
            set_grid(unused_grid, r, c, d, '.' * len(w))
        unused_letters = [c for l in unused_grid for c in l if c != '.']
        unused_letter_count = collections.Counter(unused_letters)
        makeable_words = []
        for w in unused_words:
            unused_word_count = collections.Counter(w)
            if all(unused_word_count[l] <= unused_letter_count[l] for l in unused_word_count):
                makeable_words += [w]
        lwm = sorted(makeable_words, key=len)[-1]
        lwms = [w for w in makeable_words if len(w) == len(lwm)]
    else:
        lwa = ''
        lwas = []
        lwm = ''
        lwms = []
        
    if show_anyway or len(set((len(lwp),len(lwa),len(lwm)))) == 3:
        print('\n{}'.format(fn))
        print('{} words present'.format(len(words) - len(unused_words)))
        print('Longest word present: {}, {} letters ({})'.format(lwp, len(lwp), lwps))
        print('Longest word absent: {}, {} letters ({})'.format(lwa, len(lwa), lwas))
        print('{} unused letters'.format(len([c for c in cat(cat(l) for l in g0) if c == '.'])))
        print('Longest makeable word: {}, {} ({})'.format(lwm, len(lwm), lwms))
        if show_all_makeable:
            print('All makeable words: {}'.format(makeable_words))

In [136]:
do_wordsearch_tasks('wordsearch04.txt', show_anyway=True)


wordsearch04.txt
58 words present
Longest word present: fortification, 13 letters (['fortification'])
Longest word absent: justification, 13 letters (['justification'])
57 unused letters
Longest makeable word: despairing, 10 (['despairing'])
All makeable words: ['despairing', 'gall', 'impales', 'jerkin']


In [25]:
do_wordsearch_tasks('wordsearch08.txt')


wordsearch08.txt
62 words present
Longest word present: compassionately, 15 letters (['compassionately'])
Longest word absent: retrospectives, 14 letters (['retrospectives'])
65 unused letters
Longest makeable word: vacationing, 11 (['vacationing'])


In [80]:
%%timeit
do_wordsearch_tasks_old('04-wordsearch.txt')


04-wordsearch.txt
62 words present
Longest word present: compassionately, 15 letters (['compassionately'])
Longest word absent: retrospectives, 14 letters (['retrospectives'])
65 unused letters
Longest makeable word: vacationing, 11 (['vacationing'])
All makeable words: ['albacore', 'entail', 'excreted', 'factory', 'fornicated', 'frantic', 'grease', 'grocer', 'informal', 'jockeying', 'justified', 'larynxes', 'reviewer', 'sabotage', 'unable', 'vacationing', 'widgeons']

04-wordsearch.txt
62 words present
Longest word present: compassionately, 15 letters (['compassionately'])
Longest word absent: retrospectives, 14 letters (['retrospectives'])
65 unused letters
Longest makeable word: vacationing, 11 (['vacationing'])
All makeable words: ['albacore', 'entail', 'excreted', 'factory', 'fornicated', 'frantic', 'grease', 'grocer', 'informal', 'jockeying', 'justified', 'larynxes', 'reviewer', 'sabotage', 'unable', 'vacationing', 'widgeons']

04-wordsearch.txt
62 words present
Longest word pre

In [142]:
%%timeit
do_wordsearch_tasks('wordsearch04.txt')

1 loop, best of 3: 463 ms per loop


In [143]:
do_wordsearch_tasks('example-wordsearch.txt', show_anyway=True)


example-wordsearch.txt
14 words present
Longest word present: cowhides, 8 letters (['cowhides'])
Longest word absent: adapting, 8 letters (['adapting', 'coccyxes', 'crumbled', 'molested'])
57 unused letters
Longest makeable word: shinier, 7 (['shinier'])
All makeable words: ['shinier']


In [26]:
# for fn in sorted(os.listdir()):
#     if re.match('wordsearch\d\d\.txt', fn):
#         do_wordsearch_tasks(fn)

In [27]:
width, height, grid, words = read_wordsearch('wordsearch04.txt')
for w in words:
    print(w, present(grid, w))

absolved (True, 16, 2, <Direction.right: 2>)
adorable (True, 11, 4, <Direction.right: 2>)
aeon (True, 11, 4, <Direction.down: 4>)
alias (True, 15, 15, <Direction.left: 1>)
ancestor (False, 0, 0, <Direction.left: 1>)
baritone (False, 0, 0, <Direction.left: 1>)
bemusing (False, 0, 0, <Direction.left: 1>)
blonds (False, 0, 0, <Direction.left: 1>)
bran (True, 1, 9, <Direction.left: 1>)
calcite (True, 19, 9, <Direction.upright: 6>)
candor (True, 17, 12, <Direction.right: 2>)
conciseness (False, 0, 0, <Direction.left: 1>)
consequent (False, 0, 0, <Direction.left: 1>)
cuddle (False, 0, 0, <Direction.left: 1>)
damming (True, 17, 2, <Direction.right: 2>)
dashboards (False, 0, 0, <Direction.left: 1>)
despairing (False, 0, 0, <Direction.left: 1>)
dint (False, 0, 0, <Direction.left: 1>)
dullard (True, 8, 2, <Direction.down: 4>)
dynasty (True, 3, 4, <Direction.downright: 8>)
employer (False, 0, 0, <Direction.left: 1>)
exhorts (True, 0, 8, <Direction.left: 1>)
feted (True, 5, 10, <Direction.right: 2

## Example for question text

In [28]:
import copy
grid =  [['.', '.', '.', 'p', '.', 'm', 'o', 'w', 'n', '.'], ['.', 's', 'd', 's', 'e', '.', '.', 'e', 'e', '.'], ['.', 'e', '.', 'e', 'l', 'a', 'd', '.', 'c', 'r'], ['p', 'i', '.', 'd', 't', 'i', 'r', '.', 'a', 'h'], ['r', 'z', 's', 'i', 'w', 'o', 'v', 's', 'p', 'u'], ['o', 'a', 'w', 'h', '.', 'k', 'i', 'e', 'a', 'b'], ['b', 'r', 'o', 'w', '.', 'c', '.', 'r', 'd', 'a'], ['e', 'c', 'n', 'o', 't', 'o', 'p', 's', '.', 'r'], ['d', '.', 'k', 'c', '.', 'd', '.', '.', '.', 'b'], ['.', 's', 't', 'a', 'p', 'l', 'e', '.', '.', '.']]
padded_grid =  [['f', 'h', 'j', 'p', 'a', 'm', 'o', 'w', 'n', 'q'], ['w', 's', 'd', 's', 'e', 'u', 'q', 'e', 'e', 'v'], ['i', 'e', 'a', 'e', 'l', 'a', 'd', 'h', 'c', 'r'], ['p', 'i', 'e', 'd', 't', 'i', 'r', 'i', 'a', 'h'], ['r', 'z', 's', 'i', 'w', 'o', 'v', 's', 'p', 'u'], ['o', 'a', 'w', 'h', 'a', 'k', 'i', 'e', 'a', 'b'], ['b', 'r', 'o', 'w', 'p', 'c', 'f', 'r', 'd', 'a'], ['e', 'c', 'n', 'o', 't', 'o', 'p', 's', 's', 'r'], ['d', 'i', 'k', 'c', 'h', 'd', 'n', 'p', 'n', 'b'], ['b', 's', 't', 'a', 'p', 'l', 'e', 'o', 'k', 'r']]
present_words =  ['probed', 'staple', 'rioted', 'cowhides', 'tops', 'knows', 'lived', 'rhubarb', 'crazies', 'dock', 'apace', 'mown', 'pears', 'wide']
decoy_words =  ['fickler', 'adapting', 'chump', 'foaming', 'molested', 'carnal', 'crumbled', 'guts', 'minuend', 'bombing', 'winced', 'coccyxes', 'solaria', 'shinier', 'cackles']

In [29]:
', '.join(sorted(present_words))

'apace, cowhides, crazies, dock, knows, lived, mown, pears, probed, rhubarb, rioted, staple, tops, wide'

In [92]:
sum(len(w) for w in present_words), len(present_words)

(76, 14)

In [30]:
', '.join(sorted(decoy_words))

'adapting, bombing, cackles, carnal, chump, coccyxes, crumbled, fickler, foaming, guts, minuend, molested, shinier, solaria, winced'

In [31]:
', '.join(sorted(present_words + decoy_words))

'adapting, apace, bombing, cackles, carnal, chump, coccyxes, cowhides, crazies, crumbled, dock, fickler, foaming, guts, knows, lived, minuend, molested, mown, pears, probed, rhubarb, rioted, shinier, solaria, staple, tops, wide, winced'

In [32]:
cts = collections.Counter()
for w in present_words:
    _, r, c, d = present(grid, w)
    inds = indices(grid, r, c, len(w), d)
    for i in inds:
        cts[i] += 1
    print(w, r, c, len(w), d, inds)
[i for i in cts if cts[i] > 1]

probed 3 0 6 Direction.down [(3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0)]
staple 9 1 6 Direction.right [(9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6)]
rioted 6 7 6 Direction.upleft [(6, 7), (5, 6), (4, 5), (3, 4), (2, 3), (1, 2)]
cowhides 8 3 8 Direction.up [(8, 3), (7, 3), (6, 3), (5, 3), (4, 3), (3, 3), (2, 3), (1, 3)]
tops 7 4 4 Direction.right [(7, 4), (7, 5), (7, 6), (7, 7)]
knows 8 2 5 Direction.up [(8, 2), (7, 2), (6, 2), (5, 2), (4, 2)]
lived 2 4 5 Direction.downright [(2, 4), (3, 5), (4, 6), (5, 7), (6, 8)]
rhubarb 2 9 7 Direction.down [(2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9)]
crazies 7 1 7 Direction.up [(7, 1), (6, 1), (5, 1), (4, 1), (3, 1), (2, 1), (1, 1)]
dock 8 5 4 Direction.up [(8, 5), (7, 5), (6, 5), (5, 5)]
apace 5 8 5 Direction.up [(5, 8), (4, 8), (3, 8), (2, 8), (1, 8)]
mown 0 5 4 Direction.right [(0, 5), (0, 6), (0, 7), (0, 8)]
pears 0 3 5 Direction.downright [(0, 3), (1, 4), (2, 5), (3, 6), (4, 7)]
wide 4 4 4 Direction.upright [(4, 4), (3, 5), (2

[(7, 5), (2, 3), (3, 5)]

In [33]:
do_wordsearch_tasks('example-wordsearch.txt', show_anyway=True)


example-wordsearch.txt
14 words present
Longest word present: cowhides, 8 letters (['cowhides'])
Longest word absent: molested, 8 letters (['adapting', 'coccyxes', 'crumbled', 'molested'])
27 unused letters
Longest makeable word: shinier, 7 (['shinier'])


In [34]:
g = empty_grid(10, 10)
set_grid(g, 2, 4, Direction.downright, 'lived')
print(show_grid(g))

..........
..........
....l.....
.....i....
......v...
.......e..
........d.
..........
..........
..........


In [35]:
g = empty_grid(10, 10)
set_grid(g, 2, 4, Direction.downright, 'lived')
set_grid(g, 4, 4, Direction.upright, 'wide')
set_grid(g, 9, 1, Direction.right, 'staple')
print(show_grid(g))

..........
.......e..
....l.d...
.....i....
....w.v...
.......e..
........d.
..........
..........
.staple...


In [36]:
g = empty_grid(10, 10)
set_grid(g, 8, 3, Direction.up, 'cowhides')
print(show_grid(g))

..........
...s......
...e......
...d......
...i......
...h......
...w......
...o......
...c......
..........


In [39]:
g = empty_grid(10, 10)
set_grid(g, 2, 4, Direction.downright, 'lived')
set_grid(g, 4, 4, Direction.upright, 'wide')
set_grid(g, 9, 1, Direction.right, 'staple')
set_grid(g, 8, 3, Direction.up, 'cowhides')
print(show_grid(g))

..........
...s...e..
...el.d...
...d.i....
...iw.v...
...h...e..
...w....d.
...o......
...c......
.staple...


In [37]:
# Example of word in grid that is English but isn't in the words listed in the puzzle.
g = empty_grid(10, 10)
set_grid(g, 6, 0, Direction.right, 'brow')
print(show_grid(g))

..........
..........
..........
..........
..........
..........
brow......
..........
..........
..........


In [38]:
unused_grid = copy.deepcopy(padded_grid)
for w in present_words:
    _, r, c, d = present(grid, w)
    set_grid(unused_grid, r, c, d, '.' * len(w))
print(show_grid(unused_grid))
cat(sorted(c for l in unused_grid for c in l if c in string.ascii_letters))

fhj.a....q
w....uq..v
i.a....h..
..e....i..
..........
....a.....
....p.f...
........s.
.i..h.npn.
b......okr


'aaabeffhhhiiijknnoppqqrsuvw'

In [201]:
len(sorted(c for l in unused_grid for c in l if c in 'aeiou'))

9

# All wordsearch puzzles

In [108]:
def read_all_wordsearch(fn):
    with open(fn) as f:
        text = f.read().strip()
        puzzles_text = text.split('\n\n')
        puzzles = []
        for p in puzzles_text:
            lines = p.splitlines()
            w, h = [int(s) for s in lines[0].split('x')]
            grid = lines[1:h+1]
            words = lines[h+1:]
            puzzles += [(w, h, grid, words)]
    return puzzles

In [109]:
puzzles = read_all_wordsearch('all-wordsearches.txt')
puzzles[3]

(20,
 20,
 ['esaetarotcetorpwvnma',
  'dhuejswellraqzassuzn',
  'riopstewsidftvenlnlo',
  'dltnpupodiafilgeenap',
  'yeooooosvconsgatvenm',
  'tgtonrsdtpgsungsireo',
  'csmtnlaistdklisaeyrp',
  'fguuusortrewfkrfdluo',
  'dotcnpvoyiraicsrieht',
  'drtcoataksogdaekcoyy',
  'igoialcuneoneuasirvy',
  'ajnzdpacoqrbsoshmgnt',
  'mmsxeetcewussviipeei',
  'esbrevrioprivilramsr',
  'tsgerdvcvutesbtrrska',
  'eyselimisapenheettcl',
  'ryponacqcetsdsddiouu',
  'streitsaotsedalaanlg',
  'foretselppusdfrsleae',
  'utsolacebeardingpher'],
 ['aboveboard',
  'accents',
  'applicants',
  'arbitrarily',
  'atlas',
  'bazillions',
  'bearding',
  'biathlon',
  'bivouacking',
  'canopy',
  'captivated',
  'chicory',
  'cockroach',
  'construct',
  'coups',
  'credit',
  'depreciates',
  'diameters',
  'diffuses',
  'douse',
  'downbeats',
  'dregs',
  'envy',
  'expects',
  'expurgations',
  'fact',
  'fastens',
  'festively',
  'flubbing',
  'fops',
  'fore',
  'gage',
  'gapes',
  'gawks',
  'gemstone',

In [110]:
def found_words_length(puzzle):
    width, height, grid, words = puzzle
    return sum(len(p[0]) for p in present_many(grid, words))

In [113]:
[found_words_length(p) for p in puzzles]

[309,
 335,
 338,
 346,
 364,
 372,
 337,
 340,
 353,
 371,
 343,
 348,
 375,
 343,
 363,
 376,
 347,
 363,
 342,
 348,
 377,
 342,
 355,
 351,
 342,
 331,
 362,
 354,
 323,
 353,
 337,
 340,
 349,
 362,
 361,
 350,
 348,
 327,
 370,
 362,
 334,
 333,
 341,
 354,
 355,
 358,
 355,
 358,
 357,
 351,
 351,
 346,
 326,
 332,
 352,
 347,
 346,
 369,
 363,
 361,
 365,
 364,
 359,
 352,
 344,
 352,
 348,
 360,
 333,
 352,
 374,
 360,
 349,
 343,
 360,
 345,
 364,
 355,
 349,
 349,
 355,
 328,
 358,
 344,
 335,
 339,
 365,
 328,
 343,
 350,
 340,
 342,
 342,
 357,
 362,
 333,
 357,
 346,
 341,
 348]

In [114]:
sum(found_words_length(p) for p in puzzles)

34988

In [122]:
def max_unfound_word_length(puzzle):
    width, height, grid, words = puzzle
    presences = present_many(grid, words)
    used_words = [p[0] for p in presences]
    unused_words = [w for w in words if w not in used_words]
    
    unused_grid = [[c for c in r] for r in grid]
    for w, r, c, d in presences:
        set_grid(unused_grid, r, c, d, '.' * len(w))
    unused_letters = [c for l in unused_grid for c in l if c != '.']
    unused_letter_count = collections.Counter(unused_letters)
    
    makeable_words = []
    for w in unused_words:
        unused_word_count = collections.Counter(w)
        if all(unused_word_count[l] <= unused_letter_count[l] for l in unused_word_count):
            makeable_words += [w]
    lwm = max(len(w) for w in makeable_words)
    return lwm

In [123]:
[max_unfound_word_length(p) for p in puzzles]

[11,
 13,
 11,
 11,
 10,
 9,
 12,
 12,
 11,
 10,
 15,
 11,
 10,
 11,
 10,
 12,
 11,
 9,
 11,
 11,
 10,
 12,
 11,
 12,
 13,
 12,
 10,
 10,
 11,
 9,
 11,
 11,
 8,
 12,
 13,
 10,
 11,
 11,
 9,
 8,
 12,
 13,
 11,
 9,
 13,
 11,
 11,
 11,
 13,
 12,
 10,
 11,
 11,
 12,
 10,
 8,
 12,
 11,
 10,
 11,
 8,
 12,
 10,
 11,
 12,
 12,
 12,
 12,
 11,
 11,
 12,
 12,
 13,
 10,
 10,
 10,
 10,
 12,
 11,
 11,
 12,
 11,
 9,
 14,
 11,
 13,
 12,
 10,
 10,
 12,
 13,
 10,
 10,
 12,
 11,
 12,
 11,
 11,
 12,
 11]

In [121]:
sum(max_unfound_word_length(p) for p in puzzles)

1106

In [124]:
%%timeit
sum(found_words_length(p) for p in puzzles)

1 loop, best of 3: 18.8 s per loop


In [125]:
%%timeit
sum(max_unfound_word_length(p) for p in puzzles)

1 loop, best of 3: 18.7 s per loop


In [190]:
do_wordsearch_tasks('huge-wordsearch.txt', show_anyway=True)


huge-wordsearch.txt
1149 words present
Longest word present: hypersensitivities, 18 letters (['hypersensitivities'])
Longest word absent: rambunctiousness, 16 letters (['rambunctiousness'])
57 unused letters
Longest makeable word: rambunctiousness, 16 (['rambunctiousness'])


In [191]:
%%timeit
do_wordsearch_tasks('huge-wordsearch.txt')

1 loop, best of 3: 1min 4s per loop


In [192]:
width, height, grid, words = read_wordsearch('huge-wordsearch.txt')
pm = present_many(grid, words)
pold = [w for w in words if present(grid, w)[0]]

In [193]:
len(pm)

1149

In [194]:
len(pold)

1149

In [195]:
pm_extra = [p for p in pm if p not in pold]
len(pm_extra)

1149

In [196]:
list(pm)[:3]

[('poltroons', 62, 65, <Direction.downleft: 7>),
 ('dogged', 7, 45, <Direction.down: 4>),
 ('activist', 51, 35, <Direction.downright: 8>)]

In [197]:
pold[:3]

['abound', 'abstracted', 'accidents']

In [198]:
collections.Counter(p[0] for p in pm).most_common(10)

[('retreading', 1),
 ('disavows', 1),
 ('finals', 1),
 ('conniver', 1),
 ('warding', 1),
 ('melon', 1),
 ('paging', 1),
 ('booties', 1),
 ('civilises', 1),
 ('hugged', 1)]

In [199]:
[p for p in pm if p[0] == 'seen']

[]